In [ ]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored
import asyncio

In [ ]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

In [ ]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

In [ ]:
listeners = []

# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")
    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))



connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

agent_controller.register_listeners(listeners,defaults=True)

## Create Invite Before Requesting Mediation



In [ ]:
unmediated_invitation = await agent_controller.connections.create_invitation()

## Make Connection with Mediator

In [ ]:
mediator_invitation ={'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '225a7d6c-4e78-44e4-833a-6913ccfddd23', 'serviceEndpoint': 'http://f3a685129c04.ngrok.io', 'label': 'MediatorAgent', 'recipientKeys': ['GG2wAwZxEyRkh5UtayVYfJRq1vw5MjXZyj1pjcb6knzz']}

In [ ]:
response = await agent_controller.connections.receive_invitation(mediator_invitation)

In [ ]:
connection_id = response["connection_id"]
print(connection_id)

### Request mediation

In [ ]:
### check state of connection
connection = await agent_controller.connections.get_connection(connection_id)

# Let's check for the state
if connection['state'] != 'active':
    print("No active connection. \n Please go back and ensure you have established an active connection between the mediator agent and Alice's subwallet agent")    
else:
    ## request mediation
    mediation_req = await agent_controller.mediation.request_mediation(connection_id)
    print(mediation_req)
    


## Check Mediation Records

State should be "granted" if mediator agent has accepted the mediation request

In [ ]:
### To have a look at the mediation records and we should see our mediation in there
response = await agent_controller.mediation.get_mediation_records()
print(response)
print(f"State : {response[0]['state']}")

## Set Default Mediator

Your agent will automatically use this mediator when creating connections

In [ ]:
default_mediation_res = await agent_controller.mediation.set_default_mediator(response[0]['mediation_id'])
print(default_mediation_res)

## Create Mediated Connection Invitation

In [ ]:
mediated_conn_invitation = await agent_controller.connections.create_invitation()


## See the Difference Between Invitations

The mediated connection invitation uses the routing keys and service endpoint provided by the mediator.

In [ ]:
mediated_routing_keys = mediated_conn_invitation["invitation"]["routingKeys"]
mediated_endpoint = mediated_conn_invitation["invitation"]["serviceEndpoint"]
unmediated_routing_key = unmediated_invitation["invitation"]["routingKeys"]
unmediated_endpoint = unmediated_invitation["invitation"]["serviceEndpoint"]

print("Before Mediation")
print(f"Endpoint : {unmediated_endpoint}, Routing Keys : {unmediated_routing_key}")
print("After Mediation")
print(f"Endpoint : {mediated_endpoint}, Routing Keys : {mediated_routing_keys}")